### processing the data

In [1]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer

In [2]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [3]:
from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")

raw_datasets

Reusing dataset glue (C:\Users\codenamewei\.cache\huggingface\datasets\glue\mrpc\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


DatasetDict({
    train: Dataset({
        features: ['idx', 'label', 'sentence1', 'sentence2'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['idx', 'label', 'sentence1', 'sentence2'],
        num_rows: 408
    })
    test: Dataset({
        features: ['idx', 'label', 'sentence1', 'sentence2'],
        num_rows: 1725
    })
})

# Reveal information of each column

In [4]:
raw_datasets["test"].features

{'idx': Value(dtype='int32', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None),
 'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None)}

### Get a glimpse of the dataset

In [5]:
raw_datasets["train"][4]

{'idx': 4,
 'label': 1,
 'sentence1': 'The stock rose $ 2.11 , or about 11 percent , to close Friday at $ 21.51 on the New York Stock Exchange .',
 'sentence2': 'PG & E Corp. shares jumped $ 1.63 or 8 percent to $ 21.03 on the New York Stock Exchange on Friday .'}

# Tokenize the dataset

In [6]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True) #padding = True)

Note that we’ve left the padding argument out in our tokenization function for now. This is because padding all the samples to the maximum length is not efficient: it’s better to pad the samples when we’re building a batch, as then we only need to pad to the maximum length in that batch, and not the maximum length in the entire dataset. This can save a lot of time and processing power when the inputs have very variable lengths! 

In [7]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched = True)

100%|██████████| 2/2 [00:00<00:00, 28.86ba/s]


In [8]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### To train the model , first define TrainingArguments class that will contain all the hyperparameters the Trainer will use for training and evalution. 

In [13]:
training_args = TrainingArguments(output_dir = "training-output/test-trainer-gpu")

In [14]:
trainer = Trainer(
    model, 
    training_args, 
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator, 
    tokenizer = tokenizer)# can skip this since passed in data_collator

In [15]:
trainer.train()

  1%|          | 10/1377 [00:00<01:46, 12.83it/s]

{'loss': 0.1961, 'learning_rate': 4.963689179375454e-05, 'epoch': 0.02}



  1%|          | 12/1377 [00:01<05:09,  4.41it/s]

{'eval_loss': 0.576067328453064, 'eval_runtime': 0.8483, 'eval_samples_per_second': 480.961, 'epoch': 0.02}


  1%|▏         | 20/1377 [00:02<02:31,  8.94it/s]

...

{'loss': 0.084, 'learning_rate': 3.8017429193899786e-05, 'epoch': 0.72}



 24%|██▍       | 331/1377 [00:56<04:20,  4.02it/s]

{'eval_loss': 0.9185711741447449, 'eval_runtime': 0.8978, 'eval_samples_per_second': 454.45, 'epoch': 0.72}


 25%|██▍       | 340/1377 [00:57<02:06,  8.18it/s]

{'loss': 0.007, 'learning_rate': 3.7654320987654326e-05, 'epoch': 0.74}


cpu training
```
TrainOutput(global_step=1377, training_loss=0.3178706283403118, metrics={'train_runtime': 863.3071, 'train_samples_per_second': 1.595, 'total_flos': 141940900890768.0, 'epoch': 3.0})
```

gpu training
```
TrainOutput(global_step=1377, training_loss=0.464037926368464, metrics={'train_runtime': 118.9554, 'train_samples_per_second': 11.576, 'total_flos': 141940900890768.0, 'epoch': 3.0})
```